In [1]:
import torch
import torch.nn as nn
import torch.optim as optim


In [17]:
X1 = torch.rand(50,10) + 10
X2 = torch.rand(50,10) 
X = torch.cat((X1,X2),0)

Y1 = torch.randint(1,2,(50,1))
Y2 = torch.randint(0,1,(50,1))
Y = torch.cat((Y1,Y2),0).reshape(-1)


In [77]:
# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, 1)  # Output size is 1 for binary classification

    def forward(self, x):
        out = torch.sigmoid(self.linear(x))  # Sigmoid activation for binary classification
        return out
    
# Instantiate the model
input_size = 58  # Number of features
model = LogisticRegression(input_size)

# Binary cross-entropy loss function
criterion = nn.BCELoss()

# Stochastic Gradient Descent optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training the model
num_epochs = 1000000
best_val_loss = float('inf')
best_model_path = '/mnt/mount_zc_NAS/Deepstrain/HFpEF_analysis/models/best_model.pt'  # Path to save the best model

In [79]:
# shuffle X and Y
idx = torch.randperm(50)
print(idx)
X = X[idx,:]
Y = Y[idx]


X_train = X[0:48,:]
Y_train = Y[0:48]

X_val = X[48:,:]
Y_val = Y[48:]
print(Y_val)
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)

    # Calculate loss
    loss = criterion(outputs.view(-1), Y_train.float())  # BCELoss expects float inputs

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Save the best model based on validation loss
    # if loss.item() < best_val_loss and epoch % 2000 == 0:
    #     best_val_loss = loss.item()
    #     torch.save(model.state_dict(), best_model_path)
    
    if (epoch+1) % 1000 == 0:
        accuracy, sensitivity, specificity = val(model, X_train, Y_train)
        val_accuracy, val_sensitivity,val_specificity = val(model, X_val, Y_val)

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}', ' train acc, sen, spec: ',accuracy, sensitivity, specificity, ' val acc, sen, spec: ',val_accuracy, val_sensitivity,val_specificity )
        
    

tensor([45, 21, 31, 12, 23, 13, 49, 47, 16, 39, 29, 28,  0, 44, 22,  1, 11,  5,
        19, 37, 27, 35, 10, 20, 36, 18, 25, 34, 42,  7, 15,  2, 24, 43, 32, 17,
         8, 48, 46, 33, 41, 30,  9, 40,  4, 14,  6,  3, 26, 38])
tensor([0., 1.])
Epoch [1000/1000000], Loss: 1.717844009399414  train acc, sen, spec:  0.625 1.0 0.3076923076923077  val acc, sen, spec:  1.0 1.0 1.0
Epoch [2000/1000000], Loss: 1.5507246255874634  train acc, sen, spec:  0.625 0.9545454545454546 0.34615384615384615  val acc, sen, spec:  1.0 1.0 1.0
Epoch [3000/1000000], Loss: 1.4321078062057495  train acc, sen, spec:  0.6458333333333334 0.9545454545454546 0.38461538461538464  val acc, sen, spec:  1.0 1.0 1.0
Epoch [4000/1000000], Loss: 1.3278610706329346  train acc, sen, spec:  0.6041666666666666 0.9545454545454546 0.3076923076923077  val acc, sen, spec:  1.0 1.0 1.0
Epoch [5000/1000000], Loss: 1.2444707155227661  train acc, sen, spec:  0.6041666666666666 0.9545454545454546 0.3076923076923077  val acc, sen, spec:  

KeyboardInterrupt: 

In [60]:
torch.save(model.state_dict(), best_model_path)

In [80]:
# prediction
loaded_model = LogisticRegression(input_size)
loaded_model.load_state_dict(torch.load(best_model_path))

patient_id_list1 = patient_id_list[idx]

def val(loaded_model, X, Y):
    y_pred = []
    y_pred_float = []

    for index in range(0,X.shape[0]):
        new_data = X[index,:]

        # Put the model in evaluation mode
        loaded_model.eval()

        with torch.no_grad():
            prediction = loaded_model(new_data)
            y_pred_float.append(prediction.item())
            predicted_class = 1 if prediction.item() > 0.5 else 0
            y_pred.append(predicted_class)
            # print('patient id: ',patient_id_list1[index]  , f'Predicted class: {predicted_class}, Probability: {prediction.item()}', ' ground truth: ', Y[index], 'correct?: ', predicted_class == Y[index])

    y_pred = np.asarray(y_pred)
    y_pred_float = np.asarray(y_pred_float)
    y_true = torch.clone(Y).numpy()

    # calculate accuracy using predict_collect and ground truth
    accuracy = np.sum(y_pred == y_true) / len(y_true)
    # print('accuracy: ', accuracy)
    # also calculate sensitivity and specificity, write the code please
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))

    # print('true positive: ', TP, 'true negative: ', TN, 'false positive: ', FP, 'false negative: ', FN)

    # Sensitivity (True Positive Rate)
    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0.0

    # Specificity (True Negative Rate)
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0.0
    # print('sensitivity: ', sensitivity, 'specificity: ', specificity)

    # # use y_pred_float and y_true to calculate AUC
    # from sklearn.metrics import roc_auc_score
    # auc = roc_auc_score(y_true, y_pred_float)
    # # print(auc)
    return accuracy, sensitivity, specificity

accuracy, sensitivity, specificity = val(loaded_model, X_train, Y_train)


In [73]:
# load 
import sys
sys.path.append("../../")
import pandas as pd
import os
import numpy as np
import DeepStrain.functions_collection as ff

main_path = '/mnt/mount_zc_NAS/Deepstrain'
data_path = '/mnt/mount_zc_NAS/HFpEF/data/HFpEF_data'
spreadsheet = pd.read_excel(os.path.join(data_path, 'Patient_list', 'Important_HFpEF_Patient_list_unique_patient_w_readmission_finalized.xlsx' ))

patient_list = spreadsheet.iloc[0:50]
patient_id_list = patient_list['OurID']
patient_id_list = torch.from_numpy(patient_id_list.to_numpy())



In [56]:
X = []
for patient_index in range(0,patient_list.shape[0]):
    patient_id_num = patient_list['OurID'].iloc[patient_index]
    patient_id = ff.XX_to_ID_00XX(patient_id_num)

    # load strain first:
    strain_folder = os.path.join(main_path, 'results/strain', patient_id)
    # global strain
    global_strain = np.load(os.path.join(strain_folder, 'global_strain.npy'))
    global_radial_strain = np.asarray(global_strain[0]).reshape(1,-1)
    global_circumferential_strain = np.asarray(global_strain[1]).reshape(1,-1)

    # layer strain
    layer_strain = np.load(os.path.join(strain_folder, 'layer_strain.npy'))
    layer_radial_strain = layer_strain[0,:].reshape(1,-1)
    layer_circumferential_strain = layer_strain[1,:].reshape(1,-1)

    # polar strain
    polar_strain = np.load(os.path.join(strain_folder, 'polar_strain.npy'), allow_pickle=True)
    Ecc_aha = np.asarray(polar_strain[5][:-1]).reshape(1,-1)
    Err_aha = np.asarray(polar_strain[7][:-1]).reshape(1,-1)

    # WTCI
    wtci = np.load(os.path.join(strain_folder, 'wtci.npy'), allow_pickle=True)
    wtci_aha = np.asarray(wtci[4][:-1]).reshape(1,-1)

    # load geometry then:
    geometry_folder = os.path.join(main_path, 'results/geometry', patient_id)
    # circular index
    circular_index = np.load(os.path.join(geometry_folder, 'circular_index_collect.npy')).reshape(1,-1)
    # centers
    centers = np.load(os.path.join(geometry_folder, 'centers_collect.npy'), allow_pickle=True)
    center_delta_to_base = centers[1][1:,:].reshape(1,-1)
    enclosed_area = np.asarray(centers[-1]).reshape(1,-1)

    # axis len
    axis = np.load(os.path.join(geometry_folder, 'axis_len_collect.npy'), allow_pickle=True)
    major_axis = axis[:-1,0].reshape(1,-1)
    minor_axis = axis[:-1,1].reshape(1,-1)

    # concatenate all features
    features = np.concatenate((#global_radial_strain, global_circumferential_strain, 
                            layer_radial_strain, layer_circumferential_strain, 
                            Ecc_aha, np.concatenate((Err_aha[:,0:12] , wtci_aha[:,12:16]),axis = 1),
                            # circular_index,
                            major_axis,minor_axis),
                            axis=1)
    X.append(features)
X = np.squeeze(np.asarray(X))

In [57]:
# Load Y
Y = spreadsheet['label_for_ML'].iloc[0:50]

X = torch.from_numpy(X).float()
print(X.shape, Y.shape)
Y = torch.from_numpy(Y.to_numpy()).float()

torch.Size([50, 58]) (50,)
